# Оглавление <a class='anchor' id='root'>
- [Задача 1](#1)
    - [Формулировка](#1.1)
    - [Решение](#1.2)
- [Задача 2](#2)
    - [Формулировка](#2.1)
    - [Решение](#2.2)
        - [Задание 1](#2.2.1)
        - [Задание 2](#2.2.2)

# Задача 1 <a class='anchor' id='1'>



## Формулировка <a class='anchor' id='1.1'>

Table: Transactions
|Client_id	|Report_date	|Txn_amount|
|-----------|---------------|---------|
|123	    |2017.01.01	    |50000|


Table: Rates
|Report_date |Ccy_code	|CCy_rate |
|------------|----------|---------|
|2016.12.30	 |840		|60,58    |
|2017.01.09	 |840		|61,01    |

Напишите sql запрос, который будет переводить сумму транзакций в usd (ccy_code = 840) с учетом того, что в таблице rates данные только за рабочие дни. Транзакции, совершенные в выходные, пересчитываются по курсу последнего рабочего дня перед праздником/выходным. Результат: Клиент, дата, сумма операций в usd.

[К оглавлению](#root)

## Решение <a class='anchor' id='1.2'>

Если в Sqlite3

In [ ]:
CREATE TABLE Transactions (Client_id INT, Report_date date, Txn_amount INT);
INSERT INTO Transactions VALUES (123, '2017-01-01', 50000),  (123, '2017-01-09', 50000);
CREATE TABLE Rates (Report_date date, Ccy_code INT, CCy_rate FLOAT);
INSERT INTO Rates VALUES ('2016-12-30', 840, 60.58), ('2017-01-09', 840, 61.01);

SELECT 
t.Client_id,
t.Report_date,
t.Txn_amount / (SELECT 
                    r.CCy_rate 
                FROM Rates r 
                WHERE 
                    r.Report_date<=t.Report_date 
                ORDER BY 
                    r.Report_date DESC 
                LIMIT (1)
                ) AS usd_sum
FROM Transactions t;

Если в ClickHouse

In [ ]:
CREATE TABLE RznUps.Rates (`Report_date` DATE, `Ccy_code` INT, `CCy_rate` FLOAT) ENGINE = MergeTree() PARTITION BY `Report_date` ORDER BY `Report_date`;
INSERT INTO RznUps.Rates (*) VALUES ('2016-12-30', 840, 60.58), ('2017-01-09', 840, 61.01);
CREATE TABLE RznUps.Transactions (`Client_id` INT, `Report_date` DATE, `Txn_amount` INT) ENGINE = MergeTree() PARTITION BY `Report_date` ORDER BY `Report_date`;
INSERT INTO RznUps.Transactions (*) VALUES (123, '2017-01-01', 50000); 
INSERT INTO RznUps.Transactions (*) VALUES (123, '2017-01-09', 50000);

SELECT
	t.Client_id `client_id`,
	t.Report_date `date`,
	t.Txn_amount/r.CCy_rate `usd`
FROM (SELECT 
			1 val, 
			Client_id, 
			Txn_amount, 
			Report_date 
		FROM RznUps.Transactions) t
asof LEft JOIN (SELECT 
					1 val, 
					Report_date, 
					CCy_rate, 
					Ccy_code  
				FROM RznUps.Rates) r 
ON r.val = t.val and r.Report_date <= t.Report_date

[К оглавлению](#root)

# Задача 2 <a class='anchor' id='2'>

## Формулировка <a class='anchor' id='2.1'>

Table: VSP_oper_data
|Client_id	|Report_date	|VSP_Number	|Txn_type	|Txn_amount|
|-----------|---------------|-----------|-----------|----------|
|1233455	|2017.05.02	    |1234/0123	|debit	    |10000     |

В таблице VSP_oper_data txn_type принимает значения debit, credit
Задание: 
- напишите sql запрос, который для каждого клиента выводит сумму debit, credit операций и последнее посещенное VSP по месяцам. Результат должен содержать поля: Client_id, Report_date	Debit_amount, Credit_amount, Last_VSP;
- напишите sql запрос, который для каждого клиента выведет долю debit операций клиента к debit операциям всех клиентов по месяцам. Результат должен содержать поля: Client_id, Report_date, Ratio.


[К оглавлению](#root)

## Решение <a class='anchor' id='2.2'>

### Задание 1 <a class='anchor' id='2.2.1'>

Если писать в ClickHouse

In [ ]:
CREATE TABLE RznUps.VSP_oper_data (`Client_id` INT, `Report_date` DATE, `VSP_Number` String, `Tnx_type` String, `Txn_amount` INT) ENGINE = MergeTree() PARTITION BY `Report_date` ORDER BY `Report_date`;
INSERT INTO RznUps.VSP_oper_data (*) VALUES (123, '2017-01-01', '1234/0123', 'debit','10000'), 
(123, '2017-01-03', '1234/0125', 'debit','10000'), 
(231, '2017-01-03', '1234/0125', 'debit','10000'), 
(123, '2017-01-03', '1234/0124', 'credit','10000'),
(321, '2017-01-03', '1234/0114', 'credit','10000'),
(123, '2017-01-04', '1234/0126', 'credit','5000'), 
(123, '2017-02-01', '1234/0128', 'debit','10000'),
(123, '2017-02-04', '1234/0120', 'credit','5000')

SELECT
    Client_id,
    toStartOfMonth(Report_date) Report_date, 
    sumIf(Txn_amount, Tnx_type='debit') Debit_amount,
    sumIf(Txn_amount, Tnx_type='credit') Credit_amount,
    max(v.Report_date)
FROM 
	RznUps.VSP_oper_data v
GROUP BY
    Client_id,
    Report_date 

[К оглавлению](#root)

### Задание 2 <a class='anchor' id='2.2.2'>


Если писать в ClickHouse

In [ ]:
CREATE TABLE RznUps.VSP_oper_data (`Client_id` INT, `Report_date` DATE, `VSP_Number` String, `Tnx_type` String, `Txn_amount` INT) ENGINE = MergeTree() PARTITION BY `Report_date` ORDER BY `Report_date`;
INSERT INTO RznUps.VSP_oper_data (*) VALUES (123, '2017-01-01', '1234/0123', 'debit','10000'), 
(123, '2017-01-03', '1234/0125', 'debit','10000'), 
(231, '2017-01-03', '1234/0125', 'debit','10000'), 
(123, '2017-01-03', '1234/0124', 'credit','10000'),
(321, '2017-01-03', '1234/0114', 'credit','10000'),
(123, '2017-01-04', '1234/0126', 'credit','5000'), 
(123, '2017-02-01', '1234/0128', 'debit','10000'),
(123, '2017-02-04', '1234/0120', 'credit','5000')

WITH tmp as(
	SELECT
		Client_id,
		toStartOfMonth(Report_date) Report_date, 
		sumIf(Txn_amount, Tnx_type='debit') Debit_amount_per_client
	FROM 
		RznUps.VSP_oper_data v
	GROUP BY
		Client_id,
		Report_date 
)

SELECT
	tmp.Client_id,
	tmp2.Report_date,
	ROUND(tmp.Debit_amount_per_client/tmp2.Debit_amount_per_month,5) Ratio
FROM(
	SELECT
		Report_date,
		SUM(Debit_amount_per_client) Debit_amount_per_month
	FROM tmp	
	GROUP BY 
		Report_date 
)tmp2
LEFT JOIN tmp ON tmp.Report_date = tmp2.Report_date

[К оглавлению](#root)